In [8]:
import numpy as np

# Wolves vs Prey

This code is intended to produce a simulation of wolf and prey populations over a specified time series.  The classroom discussion led to the following PRELIMINARY details about the simulation:

Here is [an interesting reference](https://www.researchgate.net/publication/239927489_Kill_Rates_and_Predation_by_Wolves_on_Ungulate_Populations_in_Bialowieza_Primeval_Forest_Poland?enrichId=rgreq-644c6d606fbbf71d082d4cbb0ade1da6-XXX&enrichSource=Y292ZXJQYWdlOzIzOTkyNzQ4OTtBUzo5ODg2MTU3NDAwMDY1M0AxNDAwNTgxODQ2ODMz&el=1_x_2&_esc=publicationCoverPdf) for wolf-prey dynamics.

**Goal:** Predict wolf and prey population N years beyond initial conditions

**Global assumptions:**
  * Isolated habitat with no human interactions
  * Restricted habitat area, forcing variable density
  * Modeling averages over single year periods (time step = 1 year)
  * No animal migration

  
**Input:**
  * Initial wolf population, assume equal male/female?  Ages?  Numbers of each.
  * Prey population, within limited species (rabbits, deer, moose, ??).
  * Animal reproduction rate: 2->N average per year (2->N is a bit arbitrary).  Does this depend on density?
  * Wolf predation rate per species (depends on population density?)
  * Habitat size (determines densities)
  * Weather adversity index (separately for wolves and prey)
  * Disease advertity index (separately for wolves and prey)
  * Average animal lifetimes

  
**Model details/assumptions:**
  * Require relationships between preditor and prey densities vs predation rate
  * Require relationship between prey density and food competition?
      * And also weather adversity?
      * Or assume general abundance?
  * No dependence of predation rate on age?
  * Require relationship between weather adversity and animal death rate
  * Require relationship between disease adversity and animal death rate
  * Require list of animal attributes & their population distribution
  * Require means to pass down traits to offspring (eg, hunting skill or reproduction rate)
  * Require model for prey density vs preditor survival rate



**Model implementation:**
  * Define class for a given animal,
      * Decorate with required variables to track animal attributes
  * Generate arrays of each species type, populated according to initial conditions
  * Iterate one year time steps
      * Calculate animal densities
      * Determine death events, remove dead animals from arrays (dead animals don't reproduce)
      * Determine birth events, adding new animals with new attributes drawn from parents
      * Repeat until desired future year

In [153]:
###################################
## Global simulation parameters:
## nWolf_init: Initial wolf population size (integer)
## nPrey_init: Initial prey population size (integer)
## wolf_age_init: Average age of initial wolf population
## prey_age_init: Average age of initial prey population
##
## wolf_repro_rate: Average number of wolf pups per year from an average wolf (integer)
## prey_repro_rate: Average number of prey offspring per year from an average animal (integer)
##
## wolf_pred_rate: Average number of prey animals each wolf consumes per year (integer)
## wolf_pred_min: Minimum number of prey animals that a wolf must consume per year to survive (integer)
##
## wolf_lifetime: Average wolf natural lifetime in years (integer)
## prey_lifetime: Average prey natural lifetime in years (integer)
##
## habitat_size: Size of habitat in square kilometers 
## weather_idx: Weather & disaster adversity index [0,1]
## disease_idx: Disease adversity index [0,1]
##
## wolf_pop_std: Standard deviation of wolf population 
###################################


## Initial population variables
nWolf_init = 5
nPrey_init = 500
wolf_age_init = 5
prey_age_init = 2
wolf_age_std = 0.5
prey_age_std = 0.5

## Reproduction rate & lifetime variables
wolf_repro_rate = 3  # Average wolf pup rate (average is 4, but we divide by 2 because we don't track gender)
prey_repro_rate = 3  # Average prey offspring rate (again, division by 2 baked in)
wolf_repro_rate_std = 0.2 # 25% standard deviation on wolf 
prey_repro_rate_std = 0.25 # 25% standard deviation

wolf_lifetime = 15 # This is the theoretical maximum
prey_lifetime = 10 # This is the theoretical maximum

## Wolf predation variables
wolf_pred_rate = 40 # number of prey each wolf consumes each year on average
wolf_pred_min = 10 # minimum number of prey a wolf must consume per year to avoid dying of hunger
wolf_pred_std = 0.25 # standard deviation of wolf prey consumption, 25%

# environment variables
habitat_size = 500 # square kilometers
weather_idx = 0.5 
disease_idx = 0.5

In [147]:
# This is the class that we'll use for an individual wolf.  It can be edited as needed.
class Wolf:
    def __init__(self,age,pred_idx,repro_idx):
        self.age = age #This is the age of the wolf, which will increment each year
        self.pred_idx = pred_idx #This is an index [0,1] to describe this wolf's predatory skill
        self.repro_idx = repro_idx #This is an index [0,1] to describe this wolf's reproduction success rate
        self.preyEaten = 0 #This is the number of prey eaten this year
    def increaseAge(self): #increment age by one year
        self.age = self.age + 1
        
# This is the class that we'll use for an individual prey animal.  It can be edited as needed.
class Prey:
    def __init__(self,age,repro_idx):
        self.age = age #This is the age of the prey animal, which will increment each year
        self.repro_idx = repro_idx #This is an index [0,1] to describe this animal's reproduction success rate
    
    def increaseAge(self): #increment age by one year
        self.age = self.age + 1
        

In [148]:
# These functions will create the initial populations of 
def initializeWolves(nWolves, avgAge):
    # create initial set of randomized wolves: newWolf = wolf(randomAge, randomPidx, randomRidx)
    # return array of wolf objects
    # set all characteristics to 0.50 randomized by the standard deviation variable above
    # should this 0.50 be default for the simulation or driven by a configurable variable?

    #array of wolves
    wolfpack = [Wolf(0,0,0) for i in range(nWolves)]
    
    # creating random wolves
    for w in range(nWolves):
        _age = np.random.normal(avgAge,wolf_age_std)
        
        # every wolf eats, setting lower bound of 10% predatory rate
        _predRate = np.random.rand()*0.9+0.1
        
        # truly random reproduction rate
        _repoRate = np.random.rand()
        
        wolfpack[w] = Wolf(_age,_predRate,_repoRate)
    return wolfpack

def initializePrey(nPrey, avgAge):
    # create initial set of randomized prey: newPrey = prey(randomAge,randomRidx)
    # return array of prey objects
    # set reproduction rate index to 0.50 randomized by the standard deviation variable above
    # should this 0.50 be default for the simulation or driven by a configurable variable?
    
    #array of wolves
    preyherd = [Prey(0,0) for i in range(nPrey)]
    
    # creating random wolves
    for p in range(nPrey):
        _age = np.random.normal(avgAge,prey_age_std)

        # prey will have a better than zero reproduction rate
        # minimum of 20%
        _repoRate = np.random.rand()*0.8+0.2
        
        preyherd[p] = Prey(_age,_repoRate)
    return preyherd

def swapLists(inList, outList):
    inList.clear()
    for instance in outList:
        inList.append(instance)
    outList = []
    return
    

In [149]:
# These functions increase and decrease the wolf and prey populations

def deathEvent(wolves,prey):
    # First simulate how many prey each wolf consumes, remove wolves that didn't consume enough prey
    # Next randomly remove prey that were eaten by wolves
    # The weather adversity index should play a role in the efficiency of wolves catching prey.
    # But does the adversity have a greater impace on the wolves or the prey?
    nEaten = 0
    for instance in wolves:
        instance.preyEaten = int(np.random.normal(wolf_pred_rate*instance.pred_idx,wolf_pred_std))
        nEaten += instance.preyEaten
    
    print("Prey eaten this year: ", nEaten, " of: ", len(prey))    
    # remove eaten prey randomly
    survivingPrey = []
    died = 0
    for instance in prey:
        _r = np.random.rand()
        if _r > 0.5 and died<nEaten:
            died += 1
            continue
        else:
            survivingPrey.append(instance)

    #replace surviving prey
    swapLists(prey,survivingPrey)
    survivingPrey = []

    #remove wolves that didn't eat enough
    survivingWolves = []
    for w in range(len(wolves)):
        if wolves[w].preyEaten >= wolf_pred_min:
            survivingWolves.append(wolves[w])
    print("Surviving wolves: ", len(survivingWolves), " of ", len(wolves))

    #replace surviving wolves
    swapLists(wolves, survivingWolves)
    survivingWolves = []

    # Death events should also be randomly applied to all animals in addition to the prey consumed by wolves
    # As an animal reaches its maximim lifetime, the chances of randomly dying should increase.
    # The random chance should include the weather and disease adversity indices.
    # Should the disease adversity index depend on population density?
    # Should the weather and disease adversity indices impact wolves and prey equally?

    # Age based death event
    for instance in wolves:
        _r = np.random.rand()
        if _r < 1-instance.age/wolf_lifetime:
            survivingWolves.append(instance)
    swapLists(wolves, survivingWolves)
    survivingWolves = []

    # Age based death event
    for instance in prey:
        _r = np.random.rand()
        if _r < 1-instance.age/prey_lifetime:
            survivingPrey.append(instance)
    swapLists(prey,survivingPrey)
    survivingPrey = []

    # Random, but rare death event
    for instance in wolves:
        _r = np.random.rand()
        if _r > 0.02:
            survivingWolves.append(instance)
    swapLists(wolves, survivingWolves)
    survivingWolves = []

    for instance in prey:
        _r = np.random.rand()
        if _r > 0.02:
            survivingPrey.append(instance)
    swapLists(prey,survivingPrey)
    survivingPrey = []

    return

def birthEvent(wolves,prey):
    # This function should come after the death event, modeling that only healthy and surviving creatures reproduce
    # Each individual should have the opportunity to give rise to offspring at a randomized rate.
    # The weather adversity index may play a role, perhaps differently for each species of animal.
    # New animals should be added to the appropriate arrays with an age of 0.
    # New animals should be initialized with characteristics that are randomly drawn from the parent's characteristics

    # each wolf gets a chance to reproduce
    for instance in wolves:
        # minimum reproduction age
        if instance.age < 1:
            continue
        nBirth = int(np.random.normal(wolf_repro_rate*instance.repro_idx/2,wolf_repro_rate_std*wolf_repro_rate))
        if nBirth < 0:
            nBirth = 0
        # ...add new prey randomly..
        for b in range(nBirth):
            wolves.append(Wolf(0,np.random.normal(instance.pred_idx,0.1),np.random.normal(instance.repro_idx,0.1)))
    
    # each prey animal gets a chance to reproduce
    for instance in prey:
        # minimum reproduction age
        if instance.age < 1:
            continue
        nBirth = int(np.random.normal(prey_repro_rate*instance.repro_idx/2,prey_repro_rate_std*prey_repro_rate))
        if nBirth < 0:
            nBirth = 0
        # ...add new prey randomly..
        for b in range(nBirth):
            prey.append(Prey(0,np.random.normal(instance.repro_idx,0.1)))
    return

def incrementAges(wolves,prey):
    # This function should just increase the age of all existing animals
    for instance in wolves:
        instance.increaseAge()
        
    for instance in prey:
        instance.increaseAge()
        
    return

In [150]:
def generateExperiment(years):
    # create our initial animal arrays
    _wolves = initializeWolves(nWolf_init, wolf_age_init)
    _prey = initializePrey(nPrey_init, prey_age_init)
    
    #now iterate on the animals
    for i in range(years):
        print("Simulating year ",i)
        incrementAges(_wolves,_prey) # increment ages first so that surviving animals age and new animals have age=0
        birthEvent(_wolves,_prey) # new animals born
        deathEvent(_wolves,_prey) # some animals die
        print("Year ", i, ", Wolves: ", len(_wolves), " , Prey: ", len(_prey))
    return _wolves, _prey

In [154]:
wolfpack, preyherd = generateExperiment(5)

Simulating year  0
Prey eaten this year:  205  of:  769
Surviving wolves:  8  of  8
Year  0 , Wolves:  7  , Prey:  475
Simulating year  1
Prey eaten this year:  379  of:  760
Surviving wolves:  12  of  13
Year  1 , Wolves:  8  , Prey:  324
Simulating year  2
Prey eaten this year:  351  of:  503
Surviving wolves:  9  of  11
Year  2 , Wolves:  8  , Prey:  205
Simulating year  3
Prey eaten this year:  512  of:  352
Surviving wolves:  13  of  13
Year  3 , Wolves:  12  , Prey:  128
Simulating year  4
Prey eaten this year:  768  of:  227
Surviving wolves:  19  of  19
Year  4 , Wolves:  16  , Prey:  92


In [155]:
print("Wolf population: ",len(wolfpack))
print("Prey population: ",len(preyherd))

if len(wolfpack)>0:
    print("Wolf 0 age: ",wolfpack[0].age)
if len(preyherd)>0:
    print("Prey 0 age: ",preyherd[0].age)

Wolf population:  16
Prey population:  92
Wolf 0 age:  3
Prey 0 age:  4
